In [1]:
import tensorflow as tf
print(tf.__version__)
import transformers
import numpy as np
import pandas as pd
import os
import re
import glob
from tqdm import tqdm
import math
import timeit

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

2022-03-08 06:44:27.989016: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-08 06:44:27.989142: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.4.1


2022-03-08 06:44:32.738625: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-08 06:44:32.741465: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-08 06:44:32.741513: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-08 06:44:32.741540: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4cae0eeb58a2): /proc/driver/nvidia/version does not exist
2022-03-08 06:44:32.744341: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [2]:
train = pd.read_csv(os.path.join("../input/feedback-prize-2021/", "train.csv"))
train[['discourse_id', 'discourse_start', 'discourse_end']] = train[['discourse_id', 'discourse_start', 'discourse_end']].astype(int)

train["discourse_len"] = train["discourse_text"].apply(lambda x: len(x.split()))
train["pred_len"] = train["predictionstring"].apply(lambda x: len(x.split()))

train_txt = glob.glob('../input/feedback-prize-2021/train/*.txt') 

cols_to_display = ['discourse_id', 'discourse_text', 'discourse_type','predictionstring', 'discourse_len', 'pred_len']
train[cols_to_display].head()

# this code chunk is copied from Rob Mulla
len_dict = {}
word_dict = {}
for t in tqdm(train_txt):
    with open(t, "r") as txt_file:
        myid = t.split("/")[-1].replace(".txt", "")
        data = txt_file.read()
        mylen = len(data.strip())
        myword = len(data.split())
        len_dict[myid] = mylen
        word_dict[myid] = myword
train["essay_len"] = train["id"].map(len_dict)
train["essay_words"] = train["id"].map(word_dict)

data_ids = train['id'].unique()

100%|██████████| 15594/15594 [00:52<00:00, 298.40it/s]


In [3]:
#initialize column
train['gap_length'] = np.nan

#set the first one
train.loc[0, 'gap_length'] = 7 #discourse start - 1 (previous end is always -1)

#loop over rest
for i in tqdm(range(1, len(train))):
    #gap if difference is not 1 within an essay
    if ((train.loc[i, "id"] == train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] > 1)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] - 2
        #minus 2 as the previous end is always -1 and the previous start always +1
    #gap if the first discourse of an new essay does not start at 0
    elif ((train.loc[i, "id"] != train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] != 0)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] -1


 #is there any text after the last discourse of an essay?
last_ones = train.drop_duplicates(subset="id", keep='last')
last_ones['gap_end_length'] = np.where((last_ones.discourse_end < last_ones.essay_len),\
                                       (last_ones.essay_len - last_ones.discourse_end),\
                                       np.nan)

cols_to_merge = ['id', 'discourse_id', 'gap_end_length']
train = train.merge(last_ones[cols_to_merge], on = ["id", "discourse_id"], how = "left")

#display an example
cols_to_display = ['id', 'discourse_start', 'discourse_end', 'discourse_type', 'essay_len', 'gap_length', 'gap_end_length']
train[cols_to_display].query('id == "AFEC37C2D43F"')

100%|██████████| 144292/144292 [00:32<00:00, 4467.04it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,discourse_start,discourse_end,discourse_type,essay_len,gap_length,gap_end_length
144270,AFEC37C2D43F,0,317,Lead,3140,NaN,NaN
144271,AFEC37C2D43F,318,515,Position,3140,NaN,NaN
144272,AFEC37C2D43F,684,692,Claim,3140,167.0,NaN
144273,AFEC37C2D43F,693,710,Claim,3140,NaN,NaN
144274,AFEC37C2D43F,714,724,Claim,3140,2.0,NaN
144275,AFEC37C2D43F,725,1360,Evidence,3140,NaN,NaN
144276,AFEC37C2D43F,1361,1471,Claim,3140,NaN,NaN
144277,AFEC37C2D43F,1472,1881,Evidence,3140,NaN,NaN
144278,AFEC37C2D43F,1882,2019,Claim,3140,NaN,NaN
144279,AFEC37C2D43F,2029,2123,Claim,3140,8.0,NaN


In [4]:
def prepare_text_data(file_name):
    with open(f"../input/feedback-prize-2021/train/{file_name}.txt") as f:
        txt = f.read()
    return [nltk.pos_tag(nltk.word_tokenize(parag)) for parag in re.split("\n\n", txt)]

def add_gap_rows(essay):
    cols_to_keep = ['discourse_start', 'discourse_end', 'discourse_type', 'gap_length', 'gap_end_length']
    df_essay = train.query('id == @essay')[cols_to_keep].reset_index(drop = True)

    #index new row
    insert_row = len(df_essay)
   
    for i in range(1, len(df_essay)):          
        if df_essay.loc[i,"gap_length"] >0:
            if i == 0:
                start = 0 #as there is no i-1 for first row
                end = df_essay.loc[0, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1
            else:
                start = df_essay.loc[i-1, "discourse_end"] + 1
                end = df_essay.loc[i, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1
    df_essay = df_essay.sort_values(by = "discourse_start").reset_index(drop=True)

    #add gap at end
    if df_essay.loc[(len(df_essay)-1),'gap_end_length'] > 0:
        start = df_essay.loc[(len(df_essay)-1), "discourse_end"] + 1
        end = start + df_essay.loc[(len(df_essay)-1), 'gap_end_length']
        disc_type = "Nothing"
        gap_end = np.nan
        gap = np.nan
        df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
        
    return(df_essay)

def prepare_train_data(essay):
    df_essay = add_gap_rows(essay)
    #code from https://www.kaggle.com/odins0n/feedback-prize-eda, but adjusted to df_essay
    essay_file = "../input/feedback-prize-2021/train/" + essay + ".txt"
    items = []
    p = 0
    with open(essay_file, 'r') as file: data = file.read()
    
    for i, row in df_essay.iterrows():
        p = int(row['discourse_start'])
        e = int(row['discourse_end'])
        items.append([data[p:e], row['discourse_type']])

    return items

In [5]:
from sklearn.metrics import accuracy_score

#Initialize tokenizer
model_name = "../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased"
bert_tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

dtype_to_num = {"Lead":0,
                "Position":1,
                "Claim":2,
                "Counterclaim":3,
                "Rebuttal":4,
                "Evidence":5,
                "Concluding Statement":6,
                "Nothing":7}

num_to_dtype = {v: k for k, v in dtype_to_num.items()}

num_class_type = 8

BERT_MAX_LENGTH = 512

In [6]:
def collect_train_sets():
    train_lines = []
    train_labels = []

    max_data_size = 6000
    max_length    = 50
    max_sentence_size = 0


    for i in tqdm(range(max_data_size)):
        for txt in prepare_train_data(data_ids[i]):
            l = len(bert_tokenizer.tokenize(txt[0]))
            if l <= max_length:
                train_lines.append(txt[0])
                train_labels.append(dtype_to_num[txt[1]])
            else:
                for t in re.split("[.?\n]", txt[0]):
                    tsize = len(bert_tokenizer.tokenize(t))
                    if tsize <= 100:
                        max_sentence_size = max([max_sentence_size, tsize])
                        train_lines.append(t)
                        train_labels.append(dtype_to_num[txt[1]])
                    else:
                        for t1 in re.split("[,]", t):
                            max_sentence_size = max([max_sentence_size, len(bert_tokenizer.tokenize(t1))])
                            train_lines.append(t1)
                            train_labels.append(dtype_to_num[txt[1]])

    max_length = max_sentence_size
    print(max_length)
    if max_length > BERT_MAX_LENGTH:
        raise ValueError("Max_length exceeds 512")
    return [train_lines, train_labels, max_length]

In [7]:
import sys
import pandas as pd

print(pd.DataFrame([[val for val in dir()], [sys.getsizeof(eval(val)) for val in dir()]],
                   index=['name','size']).T.sort_values('size', ascending=False).reset_index(drop=True))

               name       size
0             train  115308934
1         last_ones   15937203
2         word_dict     589936
3          len_dict     589936
4         train_txt     133352
..              ...        ...
63                i         28
64  BERT_MAX_LENGTH         28
65         __spec__         16
66      __package__         16
67       __loader__         16

[68 rows x 2 columns]


In [8]:
def make_train_data(datas):
    shape = (len(datas), BERT_MAX_LENGTH)
    
    input_ids = np.zeros(shape, dtype="int32")
    attention_mask = np.zeros(shape, dtype="int32")
    token_type_ids = np.zeros(shape, dtype="int32")
    
    for i, data in enumerate(datas):
        encoded = bert_tokenizer.encode_plus(datas[i],
                                             max_length=BERT_MAX_LENGTH,
                                             pad_to_max_length=True,
                                             truncation=True)
        input_ids[i] = encoded["input_ids"]
        attention_mask[i] = encoded["attention_mask"]
        token_type_ids[i] = encoded["token_type_ids"] 
    
    return [input_ids, attention_mask, token_type_ids]

def build_model():
    input_shape = (BERT_MAX_LENGTH,)
    
    input_ids = tf.keras.layers.Input(input_shape, dtype=tf.int32)
    attention_mask = tf.keras.layers.Input(input_shape, dtype=tf.int32)
    token_type_ids = tf.keras.layers.Input(input_shape, dtype=tf.int32)
    
    bert_model = transformers.TFBertModel.from_pretrained(model_name)
    
    bert_output = bert_model(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids
    )

    last_hidden_state = bert_output.last_hidden_state
    pooler_output     = bert_output.pooler_output
    
    output = tf.keras.layers.Dense(num_class_type, activation="softmax")(pooler_output)
    model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=[output])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
    return model

In [9]:
def model_train(train_lines, train_labels):
    batch_size = 256
    epoch = 30

    test_size = 15
    train_line_size = len(train_lines)

    X_train = make_train_data(train_lines[test_size:train_line_size])
    Y_train = tf.keras.utils.to_categorical(train_labels[test_size:train_line_size], num_classes=num_class_type)

    X_test  = make_train_data(train_lines[0:test_size])
    Y_test  = tf.keras.utils.to_categorical(train_labels[0:test_size], num_classes=num_class_type)
    
    model = build_model()
        
    model.summary()
    
    model.fit(
        X_train,
        Y_train,
        batch_size=batch_size,
        epochs=epoch)
    
    return model

In [10]:
train_lines, train_labels, max_length = collect_train_sets()
with tpu_strategy.scope():
    model = model_train(train_lines, train_labels)
model.save_weights("./bert_trained_model_weights.h5")

100%|██████████| 6000/6000 [04:25<00:00, 22.56it/s]
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


233


Some layers from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                

In [11]:
test_txt = glob.glob('../input/feedback-prize-2021/test/*.txt')
sub = []

for t in test_txt:
    with open(t, "r") as txt_file:
        myid          = t.split("/")[-1].replace(".txt", "")
        datas         = re.split("[.?\n]", txt_file.read())
        input_X       = make_train_data(datas)
        X_predict = model.predict(input_X)
        X_predict_num = np.argmax(X_predict, axis=1)
        p = 0
        last_found_dtype = -1
        last_word_list   = []
        for i, data in enumerate(datas):
            if len(data) == 0:
                p += 1
            else:
                word_count = len(bert_tokenizer.tokenize(data))
                if X_predict_num[i] == 7:
                    pass
                else:
                    word_list = [str(x) for x in range(p, p+word_count)]
                    if last_found_dtype == X_predict_num[i]:
                        last_word_list = last_word_list + word_list
                        sub.append((myid, num_to_dtype[X_predict_num[i]], ' '.join(last_word_list)))
                        last_word_list = []
                    else:
                        last_found_dtype = X_predict_num[i]
                        last_word_list   = word_list
                p += word_count
                
df = pd.DataFrame(sub)
df.columns = ['id','class','predictionstring']
df.to_csv('submission.csv',index=False)
df

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


,id,class,predictionstring
0,0FB0700DAF44,Evidence,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,0FB0700DAF44,Evidence,44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 5...
2,0FB0700DAF44,Evidence,70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 8...
3,0FB0700DAF44,Evidence,92 93 94 95 96 97 98 99 100 101 102 103 104 10...
4,0FB0700DAF44,Evidence,118 119 120 121 122 123 124 125 126 127 128 12...
...,...,...,...
129,D46BCB48440A,Evidence,335 336 337 338 339 340 341 342
130,D46BCB48440A,Evidence,345 346 347 348 349 350 351
131,D46BCB48440A,Evidence,352 353 354 355 356 357 358 359 360 361 362
132,D46BCB48440A,Evidence,395 396 397 398 399 400 401 402 403 404 405 40...
